# Score: 0.9919

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load  Data

In [2]:
mnist_full = pd.read_csv('./data/train.csv')
mnist_test = pd.read_csv('./data/test.csv')
mnist_full

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
mnist_full['label'].value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

In [4]:
y = mnist_full['label'].to_numpy()
y

array([1, 0, 1, ..., 7, 6, 9])

In [5]:
X = mnist_full.drop(columns='label').to_numpy().reshape(-1, 28, 28, 1)
X.shape

(42000, 28, 28, 1)

### Split Data

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [12]:
X_train.shape

(37800, 28, 28, 1)

## Build Model

In [13]:
import tensorflow as tf
from tensorflow.keras import *

In [14]:
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, (4,4), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(64, (4,4), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

Metal device set to: Apple M1


2022-03-04 23:37:27.775790: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-04 23:37:27.776067: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                             
                                                        

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [17]:
model.fit(X_train, y_train, 
          epochs=50, 
          batch_size=100, 
          validation_data=[X_val, y_val], 
          validation_batch_size=100, 
          verbose=1)

2022-03-04 23:37:44.919907: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-04 23:37:45.097499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50
374/378 [============================>.] - ETA: 0s - loss: 0.1688 - acc: 0.9483

2022-03-04 23:37:50.520524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


378/378 [==============================] - 6s 12ms/step - loss: 0.1676 - acc: 0.9486 - val_loss: 0.0817 - val_acc: 0.9714
Epoch 2/50
378/378 [==============================] - 4s 12ms/step - loss: 0.0606 - acc: 0.9811 - val_loss: 0.0581 - val_acc: 0.9807
Epoch 3/50
378/378 [==============================] - 4s 12ms/step - loss: 0.0412 - acc: 0.9871 - val_loss: 0.0563 - val_acc: 0.9845
Epoch 4/50
378/378 [==============================] - 4s 12ms/step - loss: 0.0366 - acc: 0.9885 - val_loss: 0.0412 - val_acc: 0.9881
Epoch 5/50
378/378 [==============================] - 4s 12ms/step - loss: 0.0279 - acc: 0.9915 - val_loss: 0.0476 - val_acc: 0.9869
Epoch 6/50
378/378 [==============================] - 4s 11ms/step - loss: 0.0251 - acc: 0.9918 - val_loss: 0.0521 - val_acc: 0.9876
Epoch 7/50
378/378 [==============================] - 4s 11ms/step - loss: 0.0212 - acc: 0.9931 - val_loss: 0.0489 - val_acc: 0.9867
Epoch 8/50
378/378 [==============================] - 4s 11ms/step - loss: 0.021

## Retrain on entire data

In [18]:
model.fit(X, y, epochs=5, batch_size=100)

Epoch 1/5
420/420 [==============================] - 5s 11ms/step - loss: 0.0095 - acc: 0.9975
Epoch 2/5
420/420 [==============================] - 5s 11ms/step - loss: 0.0056 - acc: 0.9984
Epoch 3/5
420/420 [==============================] - 5s 11ms/step - loss: 0.0060 - acc: 0.9981
Epoch 4/5
420/420 [==============================] - 5s 11ms/step - loss: 0.0045 - acc: 0.9986
Epoch 5/5
420/420 [==============================] - 5s 11ms/step - loss: 0.0045 - acc: 0.9986


### Predict on Test data

In [19]:
mnist_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_test = mnist_test.to_numpy().reshape(-1, 28, 28, 1)
X_test.shape

(28000, 28, 28, 1)

In [21]:
y_pred = model.predict(X_test)
y_pred

2022-03-04 23:46:30.865478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array([[5.7499256e-15, 2.1907064e-13, 1.0000000e+00, ..., 1.6235403e-12,
        1.9281825e-13, 4.9705293e-19],
       [1.0000000e+00, 9.4237723e-21, 7.4429526e-16, ..., 6.1595574e-16,
        2.1213224e-17, 9.9334353e-11],
       [6.3194137e-12, 4.7627087e-14, 6.1309507e-15, ..., 6.6596473e-13,
        2.7573605e-10, 1.0000000e+00],
       ...,
       [5.3921086e-36, 1.3441169e-29, 1.4593652e-21, ..., 3.3926052e-23,
        4.0360465e-23, 2.1631224e-28],
       [1.9946508e-10, 5.1919702e-10, 3.3104264e-12, ..., 9.4896799e-11,
        1.1734832e-08, 9.9999726e-01],
       [3.3382087e-21, 7.1704855e-19, 1.0000000e+00, ..., 2.9372876e-17,
        7.3168116e-19, 7.1820730e-28]], dtype=float32)

In [22]:
predictions = y_pred.argmax(axis=-1)
predictions

array([2, 0, 9, ..., 3, 9, 2])

## Create Submission

In [23]:
submission = pd.DataFrame({'ImageId': range(1, len(y_pred) + 1), 'Label': predictions})
submission

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [24]:
submission.to_csv('data/submission.csv', index=False)

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False

resnet_v2 = applications.ResNet50V2(include_top=False)

resnet_v2.summary()

for layer in resnet_v2.layers:
    layer.trainable = False